# Image Labeling Demo

This notebook demonstrates how to use the `ImageLabel` widget for image annotation tasks.

In [ ]:
import sys
sys.path.insert(0, '..')

from llabel import ImageLabel
import numpy as np
from PIL import Image

## Bounding Box Annotation

Annotate images with bounding boxes for object detection.

In [ ]:
# Option 1: Load from file paths
image_paths = [
    "path/to/image1.jpg",
    "path/to/image2.jpg",
    "path/to/image3.jpg",
]

# Option 2: Create sample images with NumPy
sample_images = [
    np.random.randint(0, 255, (400, 600, 3), dtype=np.uint8),
    np.random.randint(0, 255, (400, 600, 3), dtype=np.uint8),
    np.random.randint(0, 255, (400, 600, 3), dtype=np.uint8),
]

# Define object classes
classes = ["person", "car", "bicycle", "dog"]

# Create widget
widget = ImageLabel(
    images=sample_images,
    classes=classes,
    mode="bbox"
)

widget

### Instructions for Bounding Box Mode

1. Select a class from the dropdown
2. Click and drag on the image to draw a bounding box
3. Release to create the annotation
4. Use **Previous/Next** buttons or **Arrow keys** to navigate
5. Click **Clear Annotations** to remove all boxes from current image
6. Press **Esc** to cancel current drawing

## Point Annotation

Annotate specific points of interest (e.g., keypoints, landmarks).

In [ ]:
# Create sample images
point_images = [
    np.random.randint(0, 255, (500, 500, 3), dtype=np.uint8),
    np.random.randint(0, 255, (500, 500, 3), dtype=np.uint8),
]

# Define keypoint classes
keypoint_classes = ["nose", "left_eye", "right_eye", "left_ear", "right_ear"]

widget_points = ImageLabel(
    images=point_images,
    classes=keypoint_classes,
    mode="point"
)

widget_points

### Instructions for Point Mode

1. Select a keypoint class
2. Click on the image to place a point
3. Points are immediately saved
4. Clear all points with **Clear Annotations**

## Polygon Annotation

Draw polygons for segmentation tasks.

In [ ]:
# Create sample images
poly_images = [
    np.random.randint(0, 255, (400, 600, 3), dtype=np.uint8),
]

# Define segmentation classes
seg_classes = ["background", "foreground", "object"]

widget_poly = ImageLabel(
    images=poly_images,
    classes=seg_classes,
    mode="polygon"
)

widget_poly

### Instructions for Polygon Mode

1. Select a class
2. Click to place polygon vertices
3. Click near the first point to close the polygon
4. Press **Esc** to cancel current polygon
5. Polygon is saved when closed

## Working with PIL Images

In [ ]:
# Create PIL images
pil_images = [
    Image.new('RGB', (400, 300), color=(255, 0, 0)),
    Image.new('RGB', (400, 300), color=(0, 255, 0)),
    Image.new('RGB', (400, 300), color=(0, 0, 255)),
]

widget_pil = ImageLabel(
    images=pil_images,
    classes=["red", "green", "blue"],
    mode="bbox"
)

# widget_pil  # Uncomment to display

## Export Annotations

Get annotations with normalized coordinates (0-1 range).

In [ ]:
# Get raw annotations (relative coordinates)
annotations = widget.get_annotations()
print("Raw annotations (relative coordinates 0-1):")
print(annotations[0])

# Get normalized annotations (absolute pixel coordinates)
normalized = widget.get_normalized_annotations()
print("\nNormalized annotations (absolute pixels):")
print(normalized[0])

## Export to COCO Format

Convert annotations to COCO format for object detection.

In [ ]:
def export_to_coco(widget, image_width=600, image_height=400):
    """Export annotations to COCO format."""
    coco_format = {
        "images": [],
        "annotations": [],
        "categories": []
    }
    
    # Add categories
    classes = widget.classes
    for idx, cls in enumerate(classes):
        coco_format["categories"].append({
            "id": idx,
            "name": cls
        })
    
    # Get normalized annotations
    annotations = widget.get_normalized_annotations(
        image_width=image_width,
        image_height=image_height
    )
    
    ann_id = 0
    for img_idx, img_ann in enumerate(annotations):
        # Add image info
        coco_format["images"].append({
            "id": img_idx,
            "width": image_width,
            "height": image_height,
            "file_name": f"image_{img_idx}.jpg"
        })
        
        # Add annotations
        for elem in img_ann["elements"]:
            if elem["type"] == "bbox":
                x1, y1, x2, y2 = elem["coords"]
                width = x2 - x1
                height = y2 - y1
                
                coco_format["annotations"].append({
                    "id": ann_id,
                    "image_id": img_idx,
                    "category_id": elem["class_idx"],
                    "bbox": [x1, y1, width, height],
                    "area": width * height,
                    "iscrowd": 0
                })
                ann_id += 1
    
    return coco_format

# Export
coco_data = export_to_coco(widget)
print(f"Exported {len(coco_data['annotations'])} annotations")
print(f"Categories: {[c['name'] for c in coco_data['categories']]}")

## Save Annotations

In [ ]:
import json

# Save COCO format
with open('coco_annotations.json', 'w') as f:
    json.dump(coco_data, f, indent=2)

print("Saved annotations to coco_annotations.json")

## Custom Colors

Customize annotation colors for each class.

In [ ]:
# Custom colors (CSS color strings)
custom_colors = [
    "#FF0000",  # Red
    "#00FF00",  # Green
    "#0000FF",  # Blue
    "#FFFF00",  # Yellow
]

widget_custom = ImageLabel(
    images=sample_images[:1],
    classes=classes,
    colors=custom_colors,
    mode="bbox"
)

# widget_custom  # Uncomment to display